<a href="https://colab.research.google.com/github/felipeabe/artificial-neural-network/blob/projetoComCSV/trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#uploading files
from google.colab import files
uploaded = files.upload()

#importing libs
import pandas as pd

Saving houses.csv to houses.csv


In [6]:
dfb = pd.read_csv('alzheimer.csv')

# Selecionar colunas úteis
columns_to_keep = [
    'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity',
    'DietQuality', 'SleepQuality', 'Smoking',
    'FamilyHistoryAlzheimers', 'Depression', 'Hypertension', 'Diagnosis'
]

# Filtrar o DataFrame
filtered_alzheimer_data = dfb[columns_to_keep]

# Verificar valores ausentes
missing_values = filtered_alzheimer_data.isnull().sum()

# Preencher valores ausentes (se houver)
filtered_alzheimer_data.fillna(filtered_alzheimer_data.median(), inplace=True)

# Salvar em um novo CSV
cleaned_file_path = 'alzheimer_cleaned.csv'
filtered_alzheimer_data.to_csv(cleaned_file_path, index=False)

missing_values, cleaned_file_path


<ipython-input-6-cd5cf057ebfb>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_alzheimer_data.fillna(filtered_alzheimer_data.median(), inplace=True)


(Age                        0
 BMI                        0
 AlcoholConsumption         0
 PhysicalActivity           0
 DietQuality                0
 SleepQuality               0
 Smoking                    0
 FamilyHistoryAlzheimers    0
 Depression                 0
 Hypertension               0
 Diagnosis                  0
 dtype: int64,
 'alzheimer_cleaned.csv')

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separar as variáveis explicativas e alvo
X = filtered_alzheimer_data.drop(columns=['Diagnosis'])
y = filtered_alzheimer_data['Diagnosis']

# Tratar valores faltantes em X
X = X.fillna(X.mean())

# Normalizar os dados (apenas X)
scaler = MinMaxScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Dividir os dados em treino e teste (stratified split para balancear classes)
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.3, random_state=42, stratify=y
)

# Redefinir os índices para alinhamento
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Combinar novamente X e y para salvar como arquivos separados
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Salvar os arquivos
train_file_path = 'alzheimer_train.csv'
test_file_path = 'alzheimer_test.csv'

train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)

# Checar dados salvos
print("Primeiras linhas do conjunto de treino:")
print(train_data.head())

print("Primeiras linhas do conjunto de teste:")
print(test_data.head())


Primeiras linhas do conjunto de treino:
        Age       BMI  AlcoholConsumption  PhysicalActivity  DietQuality  \
0  0.300000  0.747718            0.338947          0.330269     0.377028   
1  0.866667  0.163598            0.936547          0.881996     0.036663   
2  0.866667  0.526223            0.931950          0.654257     0.130851   
3  0.066667  0.996893            0.871020          0.661547     0.967679   
4  0.300000  0.399295            0.932860          0.121732     0.465150   

   SleepQuality  Smoking  FamilyHistoryAlzheimers  Depression  Hypertension  \
0      0.496147      1.0                      0.0         0.0           0.0   
1      0.411456      0.0                      1.0         0.0           0.0   
2      0.936762      0.0                      1.0         0.0           0.0   
3      0.857746      0.0                      0.0         0.0           0.0   
4      0.661064      0.0                      0.0         0.0           0.0   

   Diagnosis  
0          0 

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Funções de ativação e suas derivadas
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

# Funções de perda
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Classe para a Rede Neural
class NeuralNetwork:
    def __init__(self, layers, activation='sigmoid', learning_rate=0.001):
        self.layers = layers
        self.learning_rate = learning_rate
        self.activation = sigmoid if activation == 'sigmoid' else relu
        self.activation_derivative = sigmoid_derivative if activation == 'sigmoid' else relu_derivative

        # Inicializar pesos e bias com He Initialization
        self.weights = [np.random.randn(layers[i], layers[i - 1]) * np.sqrt(2 / layers[i - 1]) for i in range(1, len(layers))]
        self.biases = [np.zeros((layers[i], 1)) for i in range(1, len(layers))]

    def forward(self, X):
        self.a = [X.T]  # Ativações (saídas) por camada
        self.z = []     # Soma ponderada (z = W.a + b)

        for i in range(len(self.weights)):
            z = np.dot(self.weights[i], self.a[-1]) + self.biases[i]
            self.z.append(z)
            self.a.append(self.activation(z))

        return self.a[-1].T  # Saída final da rede

    def backward(self, X, y):
        m = X.shape[0]
        y = y.reshape(-1, 1).T
        d_weights = [np.zeros_like(w) for w in self.weights]
        d_biases = [np.zeros_like(b) for b in self.biases]

        # Cálculo do erro na saída
        dz = self.a[-1] - y  # Erro na saída
        for i in reversed(range(len(self.weights))):
            dw = np.dot(dz, self.a[i].T) / m
            db = np.sum(dz, axis=1, keepdims=True) / m
            d_weights[i] = dw
            d_biases[i] = db

            if i > 0:  # Propagação do erro para trás
                dz = np.dot(self.weights[i].T, dz) * self.activation_derivative(self.z[i-1])

        # Clipping de gradientes para evitar explosão
        for i in range(len(self.weights)):
            d_weights[i] = np.clip(d_weights[i], -1, 1)
            d_biases[i] = np.clip(d_biases[i], -1, 1)

            # Atualização dos pesos e bias
            self.weights[i] -= self.learning_rate * d_weights[i]
            self.biases[i] -= self.learning_rate * d_biases[i]

    def train(self, X, y, loss='binary_cross_entropy', epochs=100):
        print(f"Treinando rede neural com {len(self.layers) - 1} camadas ocultas")
        print(f"Função de ativação: {self.activation.__name__}\n")

        for epoch in range(epochs):
            y_pred = self.forward(X)

            # Selecionar função de perda
            if loss == 'binary_cross_entropy':
                loss_value = binary_cross_entropy(y, y_pred)
            else:
                raise ValueError("Função de perda não suportada.")

            self.backward(X, y)

            if epoch % 10 == 0:
                print(f"Epoch {epoch}/{epochs}, Loss: {loss_value:.4f}")
                print(f"y_pred range: min={y_pred.min()}, max={y_pred.max()}")

    def predict(self, X):
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)

# Carregar os dados
train_data = pd.read_csv('alzheimer_train.csv')
test_data = pd.read_csv('alzheimer_test.csv')

# Separar variáveis explicativas e alvo
X_train = train_data.drop(columns=['Diagnosis']).values
y_train = train_data['Diagnosis'].values
X_test = test_data.drop(columns=['Diagnosis']).values
y_test = test_data['Diagnosis'].values

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Identificar os NaN
nan_indices = np.argwhere(np.isnan(X_train))
print(f"Índices com NaN em X_train:")
for i, j in nan_indices:
    print(f"Linha {i}, Coluna {j}")

# Substituir NaN pela média da coluna
col_means = np.nanmean(X_train, axis=0)
for i, j in nan_indices:
    X_train[i, j] = col_means[j]

# Validar novamente
print(f"X_train contém NaN? {np.isnan(X_train).any()}")


# Criar e treinar o modelo
nn = NeuralNetwork(layers=[X_train.shape[1], 10, 1], activation='relu', learning_rate=0.001)
nn.train(X_train, y_train, epochs=100)

# Avaliar o modelo
y_pred = nn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy on Test Data: {accuracy:.2%}")


Índices com NaN em X_train:
X_train contém NaN? False
Treinando rede neural com 2 camadas ocultas
Função de ativação: relu

Epoch 0/100, Loss: 6.3053
y_pred range: min=0.0, max=1.539136140729494
Epoch 10/100, Loss: 6.2974
y_pred range: min=0.0, max=1.5646255555334259
Epoch 20/100, Loss: 6.2931
y_pred range: min=0.0, max=1.5899237143295009
Epoch 30/100, Loss: 6.2854
y_pred range: min=0.0, max=1.6150250637704997
Epoch 40/100, Loss: 6.2686
y_pred range: min=0.0, max=1.639924712164534
Epoch 50/100, Loss: 6.2544
y_pred range: min=0.0, max=1.6646091545235286
Epoch 60/100, Loss: 6.2459
y_pred range: min=0.0, max=1.6890611161879217
Epoch 70/100, Loss: 6.2402
y_pred range: min=0.0, max=1.7132764194162635
Epoch 80/100, Loss: 6.2470
y_pred range: min=0.0, max=1.7372461547120202
Epoch 90/100, Loss: 6.2314
y_pred range: min=0.0, max=1.760960349994987

Accuracy on Test Data: 64.03%
